PREPROCESSING

In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
datagen = ImageDataGenerator(
        )

I = np.load('s_x_train.npy')
L = np.load('s_y_train.npy')
label_count = 0 
for x in I:
    x = x.reshape((1,) + x.shape)

    if(L[label_count] == 0):
        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir='test/nodefect', save_prefix='nodefect', save_format='jpeg'):
            break
    
    else:
        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir='test/defect', save_prefix='defect', save_format='jpeg'):
            break
    label_count+=1

Using TensorFlow backend.


TRAINING

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 256
nb_validation_samples = 40
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('pcb_defect_model.h5')

Found 256 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/50
16/16 [==============================] - 7s 424ms/step - loss: 0.7887 - accuracy: 0.5273 - val_loss: 0.7307 - val_accuracy: 0.5312
Epoch 2/50
16/16 [==============================] - 5s 317ms/step - loss: 0.6849 - accuracy: 0.5977 - val_loss: 0.6373 - val_accuracy: 0.8750
Epoch 3/50
16/16 [==============================] - 4s 250ms/step - loss: 0.6400 - accuracy: 0.6406 - val_loss: 0.6409 - val_accuracy: 0.7083
Epoch 4/50
16/16 [==============================] - 4s 257ms/step - loss: 0.5868 - accuracy: 0.7461 - val_loss: 0.3943 - val_accuracy: 0.7812
Epoch 5/50
16/16 [==============================] - 4s 253ms/step - loss: 0.5212 - accuracy: 0.7734 - val_loss: 0.2712 - val_accuracy: 0.9167
Epoch 6/50
16/16 [==============================] - 4s 249ms/step - loss: 0.4862 - accuracy: 0.7891 - val_loss: 0.5313 - val_accuracy: 0.7500
Epoch 7/50
16/16 [==============================] - 4s 254ms/step -

TESTING

In [10]:
test_data_dir = 'data/test'
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size=(img_width, img_height),batch_size=batch_size,class_mode='binary')
results = model.evaluate(test_generator,verbose=1)
print('test loss, test acc:', results)

Found 4238 images belonging to 2 classes.
265/265 [==============================] - 19s 72ms/step
test loss, test acc: [0.3725375533103943, 0.8397828936576843]


In [18]:
data = 'data/train'

from fastai.metrics import error_rate # 1 - accuracy
# learn = create_cnn(data, models.resnet34, metrics=[accuracy])
learner = cnn_learner(data, models.resnet18, metrics=[accuracy])

NameError: name 'cnn_learner' is not defined